In [ ]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn

In [1]:
#nbi:hide_in
%matplotlib inline
import spectral as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from spectrum import rank_normalizer,scale_max_min,scale_white_dark,plot_spectra,zoom
from spectrum_ml import cluster_pixels_kmeans,detect_aphid,blob_detector
from ipywidgets import interact, interactive, widgets
from IPython.display import display
import io
import os

In [ ]:
chemin = widgets.Text()

options = {
    'title': 'Chemin_spectral',
}

def chm(x):
    chemin.value = str(x)

ch = interactive(chm, x = './home/jovyan/VL1grF/capture/', options = options)

In [ ]:
display(ch)

In [ ]:
def on_button_clicked1(b):
    with output1:
        output1.clear_output()
        os.environ['KMP_DUPLICATE_LIB_OK']='True'
        os.environ['SPECTRAL_DATA'] = chemin.value
        print(chemin.value)

In [ ]:
button1 = widgets.Button(description="Rechargez l'environnement")
output1 = widgets.Output()
display(button1, output1)

button1.on_click(on_button_clicked1)

In [ ]:
wl = widgets.FileUpload(
    accept='',  # A/home/plavainne/Documents/BIM/06022020/06022020/VL1grF/ccepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description = 'Wave_Length'
    #command = ouvrir_csv()
    #value=pd.read_csv
)

display(wl)

In [ ]:
# Create text widget for output
exp_name1 = widgets.Text()
exp_name2 = widgets.Text()
exp_name3 = widgets.Text()

In [ ]:
!pwd

In [ ]:
options1 = {
    'title': 'Exp_name',
}

options2 = {
    'title': 'Dark Ref',
}

options3 = {
    'title': 'White Ref',
}

In [ ]:
# Define function to bind value of the input to the output variable 
def name1(x):
    exp_name1.value = str(x)

def name2(x):
    exp_name2.value = str(x)
    
def name3(x):
    exp_name3.value = str(x)

In [ ]:
en1 = interactive(name1, x = "VL1grF", options = options1)
en2 = interactive(name2, x = "DARKREF_VL1grF", options = options2)
en3 = interactive(name3, x = "WHITEREF_VL1grF", options = options3)

In [ ]:
display(en1, options = options1)

In [ ]:
display(en2, options = options2)

In [ ]:
display(en3, options = options3)

In [ ]:
def on_button_clicked2(b):
    with output2:
        
        output2.clear_output()
        wave_lengths = pd.read_csv(io.BytesIO(wl.value[list(wl.value.keys())[0]]['content']), header=None, index_col=0)
        image_path = exp_name1.value + '.hdr'#current_dir.value+exp_name1.value+'/capture/'+exp_name1.value+'.hdr'
        dark_ref_path = exp_name2.value + '.hdr'#current_dir.value+exp_name.value+'/capture/DARKREF_'+exp_name.value+'.hdr'
        white_ref_path = exp_name3.value + '.hdr'#current_dir.value+exp_name.value+'/capture/WHITEREF_'+exp_name.value+'.hdr'
        
        image_name = exp_name1.value + '.raw'#current_dir.value+exp_name1.value+'/capture/'+exp_name1.value+'.hdr'
        dark_ref_name = exp_name2.value + '.raw' #current_dir.value+exp_name.value+'/capture/DARKREF_'+exp_name.value+'.hdr'
        white_ref_name = exp_name3.value + '.raw' #current_dir.value+exp_name.value+'/capture/WHITEREF_'+exp_name.value+'.hdr'
        
        img = sp.envi.open(image_path)

        dark_ref = sp.open_image(dark_ref_path)#, dark_ref_name)
        dark_spectrum = dark_ref.asarray().reshape(-1,dark_ref.shape[2]).mean(axis=0)

        white_ref = sp.open_image(white_ref_path)#, white_ref_name)
        white_spectrum = white_ref.asarray().reshape(-1,white_ref.shape[2]).mean(axis=0)
        
        img_norm = scale_white_dark(img.asarray(),dark_spectrum=dark_spectrum,white_spectrum=white_spectrum)
        img_norm = scale_max_min(img_norm)
        
        img_norm_reshape = img_norm.reshape(-1,img_norm.shape[-1])
        
        print(img.shape)
        plot_spectra(img,  wave_lengths=(29,113,227), grid_step=20)
        plt.show()
        
        pixels_sample = img_norm_reshape[np.random.choice(img_norm_reshape.shape[0], 5000, replace=False)]
        km = cluster_pixels_kmeans(pixels_sample,10)
        
        c = km.predict(img_norm_reshape)
        sns.heatmap(c.reshape(img_norm.shape[:2]),cmap='Paired')
        plt.show()
        
        cluster_scores = detect_aphid(km.cluster_centers_,
                 model_files=["ML_models/model_vae_1Dconv"],
                 svm_model_pf='ML_models/SVM_pf_rank.model',
                 svm_model_p='ML_models/SVM_p_rank.model')
        
        best_cluster = np.argmax(cluster_scores)
        ############################################
        # Changer le cluster si besoin
        #best_cluster = 0
        ############################################
        print(best_cluster)
        
        
        plt.plot(cluster_scores,"o")
        plt.xlabel("Cluster ID")
        plt.ylabel("Aphid Score")
        plt.show()
        
        sns.heatmap((c==best_cluster).reshape(img_norm.shape[:2]))
        plt.show()
        
        chosen_pixels = img_norm_reshape[(c==best_cluster)]
        scores_aphids = detect_aphid(chosen_pixels,
                             model_files=["ML_models/model_vae_1Dconv"],
                             svm_model_pf='ML_models/SVM_pf_rank.model',
                             svm_model_p='ML_models/SVM_p_rank.model')
        
        sns.violinplot(scores_aphids)
        plt.xlabel("Aphid score")
        plt.show()
        
        cc = pd.Series(c.copy())
        cc[c!=best_cluster] = 0
        cc[c==best_cluster] = 1
        cc[c==best_cluster] *= scores_aphids
        #cc2 = cc >0.7#> np.percentile(cc,99.9)
        
        sns.heatmap(cc.values.reshape(img_norm.shape[:2]))
        #sns.heatmap(cc2.values.reshape(img_norm.shape[:2]))
        plt.show()


In [ ]:
button2 = widgets.Button(description="Rechargez les donnees")
output2 = widgets.Output()
display(button2, output2)

button2.on_click(on_button_clicked2)

In [9]:
from tkinter import Button
from tkinter import Label
from tkinter import StringVar
from tkinter import Tk
from tkinter.filedialog import askopenfilename
import os, os.path

 
def on_button_clicked3(b):
    with output3:
        !echo ${DISPLAY}
        !grep "# AutomaticLogin" /etc/gdm3/custom.conf >/dev/null && export ${DISPLAY}=":1" || export ${DISPLAY}=":0"
        #os.environ['${DISPLAY}']
        print(os.environ.items())
        os.environ['QT_QPA_PLATFORM']='offscreen'
        os.environ['${DISPLAY}']= ':0.0'
        
        !export ${DISPLAY}=:0.0
        
        if os.environ.get('${DISPLAY}','') == '':
            print('no display found. Using :0.0')
            os.environ.__setitem__('${DISPLAY}', ':0.0')
  
        FILETYPES = [ ("All files", "*") ]



        root = Tk()

        filename = StringVar(root)



        label = Label(root, textvariable=filename)
        label.pack()



         #def set_filename():
            #filename.set(sp.open_image(askopenfilename(filetypes=FILETYPES)))
            #print(filename)
            #dark_ref = sp.open_image(askopenfilename(filetypes=FILETYPES))
            #filename.set(askopenfilename(filetypes=FILETYPES))
            

        #button = Button(root, text='Open', command=set_filename)
        #button.pack()
        dark_ref = sp.open_image(askopenfilename(filetypes=FILETYPES))
        root.mainloop()
        
        print(dark_ref)


In [10]:
button3 = widgets.Button(description="Rechargez les donnees")
output3 = widgets.Output()
display(button3, output3)

button3.on_click(on_button_clicked3)

Button(description='Rechargez les donnees', style=ButtonStyle())

Output()

In [ ]:
os.environ['DISPLAY']